# Avazu - Click-Through Rate Prediction

Julia Kang - jkang613@gmail.com - AMLI 2019 - Mills College

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import linear_model
# from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from math import sqrt #for RMSE
  
import tensorflow as tf 
from tensorflow.feature_column import numeric_column,categorical_column_with_hash_bucket,categorical_column_with_vocabulary_list
from tensorflow.estimator import LinearClassifier
from tensorflow import keras
from tensorflow.data import Dataset

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Model: Multilayer Perception Neural Network

In [1]:
# raw_df = pd.read_csv('numeric_rand_sample.csv')
raw_df = pd.read_csv('numeric_rand_sample.csv')
raw_df = raw_df.drop('Unnamed: 0', axis = 1) 

raw_df.head() #ad id is unique to each row
# len(sub_df)

NameError: name 'pd' is not defined

In [14]:
FEATURES = []

for col in raw_df.columns:
    if np.issubdtype(raw_df[col].dtypes , np.number):
        FEATURES.append(col)
FEATURES.remove('click')
TARGET = 'click'

In [15]:
# split the data into test and train
train, test  = train_test_split(
  raw_df,                        # split our digits dataframe
  stratify=raw_df[TARGET],       # stratify by the 'click' column   
  test_size=0.2,                # 20% of the data should be held for testing
  random_state=42               # hard-coded random state for repeatability in the example
)

train.head()

,click,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,...,time,device_type_a,device_type_b,device_type_c,device_type_d,device_type_e,device_conn_type_a,device_conn_type_b,device_conn_type_c,device_conn_type_d
242327,0,1005,0,1145,1680,5,2294,75,3,43137,...,4,0,1,0,0,0,1,0,0,0
274613,1,1005,1,1854,1327,19,2129,65,0,43137,...,3,0,1,0,0,0,1,0,0,0
323457,0,1005,0,263,2056,1,2129,65,0,43137,...,0,0,1,0,0,0,1,0,0,0
275743,0,1005,1,837,180,19,2129,65,0,43137,...,4,0,1,0,0,0,1,0,0,0
394154,0,1005,1,284,1895,19,2129,65,0,43137,...,16,0,1,0,0,0,1,0,0,0


In [16]:
feature_columns = []

for col in FEATURES:
    feature_columns.append(numeric_column(col, dtype=tf.dtypes.int64))


In [17]:
#train
train_tf_x = []
for col in FEATURES:
    train_tf_x.append(train[col])
    
train_x = np.asarray(train_tf_x)
train_x = np.transpose(train_x)

train_y = np.asarray(train['click'])

#train
test_tf_x = []
for col in FEATURES:
    test_tf_x.append(test[col])
    
test_x = np.asarray(test_tf_x)
test_x = np.transpose(test_x)

test_y = np.asarray(test['click'])

print('Training features:', train_x.shape)
print('Training labels:', train_y.shape)
print('Test features:', test_x.shape)
print('Test labels:', test_y.shape)

Training features: (323528, 33)
Training labels: (323528,)
Test features: (80882, 33)
Test labels: (80882,)


In [18]:
# Define the model by specifying a neural network layers

# LOOK INTO CLASS WEIGHTS

model = tf.keras.Sequential([
    
    #input layer 1
    tf.keras.layers.Dense(19,input_shape=(len(FEATURES),),activation=tf.nn.tanh),
    
    # Layer 2
#     tf.keras.layers.Dense(16, activation=tf.nn.sigmoid),
    
    # Layer 2
    tf.keras.layers.Dense(12, activation=tf.nn.tanh),
    
    # output
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid) 
])

# Set model's settings, including loss function, optimizer, and metrics

model.compile(
    optimizer = 'adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [19]:
model.fit(train_x, train_y,epochs=10, batch_size=10000, shuffle=True)

# loss: 0.4559 - acc: 0.8300 - when using all sigmoids and 4 layers
# loss: 0.4698 - acc: 0.8300 - when using all sigmoid and decreasing number of nodes
# loss: 0.4584 - acc: 0.8300 - when using all sigmoid and there is only 3 layers 

Epoch 1/9
323528/323528 [==============================] - 1s 4us/sample - loss: 0.5165 - acc: 0.7836
Epoch 2/9
323528/323528 [==============================] - 1s 2us/sample - loss: 0.4617 - acc: 0.8300
Epoch 3/9
323528/323528 [==============================] - 1s 2us/sample - loss: 0.4584 - acc: 0.8300
Epoch 4/9
323528/323528 [==============================] - 1s 2us/sample - loss: 0.4563 - acc: 0.8300
Epoch 5/9
323528/323528 [==============================] - 1s 2us/sample - loss: 0.4552 - acc: 0.8300
Epoch 6/9
323528/323528 [==============================] - 1s 2us/sample - loss: 0.4547 - acc: 0.8300
Epoch 7/9
323528/323528 [==============================] - 1s 2us/sample - loss: 0.4542 - acc: 0.8300
Epoch 8/9
323528/323528 [==============================] - 1s 2us/sample - loss: 0.4539 - acc: 0.8300
Epoch 9/9
323528/323528 [==============================] - 1s 2us/sample - loss: 0.4535 - acc: 0.8300
